In [ ]:
# default_exp engine

# Engine

> Use a chess engine to analyze moves.

In [ ]:
#export
import chess.engine
import chess.pgn
import pandas as pd

In [ ]:
stockfish = "stockfish_14_linux_x64_popcnt/stockfish_14_x64_popcnt" # location of stockfish executable or alias

In [ ]:
engine = chess.engine.SimpleEngine.popen_uci(stockfish)

In [ ]:
pgn = open("data/magnus.pgn")

In [ ]:
game = chess.pgn.read_game(pgn)

In [ ]:
game.headers

Headers(Event='Rated Blitz game', Site='https://lichess.org/lAV0T0zl', Date='2021.12.23', Round='?', White='DrNykterstein', Black='may6enexttime', Result='1-0', BlackElo='2974', BlackRatingDiff='-2', BlackTitle='GM', ECO='B20', Termination='Normal', TimeControl='180+0', UTCDate='2021.12.23', UTCTime='23:28:07', Variant='Standard', WhiteElo='3212', WhiteRatingDiff='+2', WhiteTitle='GM')

In [ ]:
#export
def evaluate_game(game, engine, limit=chess.engine.Limit(time=.1)):
    """Go through a game move-by-move and evaluate each position with an engine.
    """
    board = game.board()
    moves = game.mainline_moves()
    evals = []
    for move in moves:
        board.push(move)
        info = engine.analyse(board, limit=limit)
        score = info["score"].pov(chess.WHITE).score(mate_score=10_000)
        evals.append(score)
    return evals

In [ ]:
evals = evaluate_game(game, engine)

In [ ]:
evals

[28,
 46,
 -20,
 -20,
 -12,
 -9,
 33,
 67,
 27,
 20,
 -116,
 -95,
 -55,
 -50,
 -46,
 -67,
 -118,
 -136,
 -159,
 -138,
 -144,
 -141,
 -123,
 -16,
 -70,
 -62,
 -59,
 -49,
 -81,
 -57,
 -70,
 -42,
 -27,
 8,
 -21,
 10,
 18,
 0,
 0,
 319,
 350,
 375,
 330,
 275,
 261,
 347,
 379,
 559,
 517,
 525,
 528,
 548,
 554,
 617,
 610]

In [ ]:
#export
def evaluate_pgn(pgn, engine, limit=chess.engine.Limit(time=.1)):
    """Evaluate all games in a PGN with the engine.
    """
    game = chess.pgn.read_game(pgn)
    all_evals = []
    while game:
        evals = evaluate_game(game, engine, limit)
        all_evals.append(evals)
        game = chess.pgn.read_game(pgn)
    df = pd.DataFrame(all_evals)
    return df

In [ ]:
pgn = open("data/magnus.pgn")

In [ ]:
evals = evaluate_pgn(pgn, engine)

In [ ]:
evals.head()

,0,1,2,3,4,5,6,7,8,9,...,127,128,129,130,131,132,133,134,135,136
0,26,35,-23,-11,-8,0,15,62,25,54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31,32,38,44,24,25,38,26,38,26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,43,33,-19,-25,-7,-3,-9,9,24,15,...,9986.0,9984.0,9989.0,9992.0,9993.0,9994.0,9994.0,9988.0,9989.0,9990.0
3,33,40,44,49,28,17,5,8,15,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,44,42,-33,-16,-18,-7,-21,-19,-37,-27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
evals.to_csv("data/magnus_evals.csv")